# Use Model Context Protocol (MCP) as tools with Strands Agent

## Overview
The [Model Context Protocol (MCP)](https://modelcontextprotocol.io/introduction) is an open protocol that standardizes how applications provide context to Large Language Models (LLMs). Strands AI SDK integrates with MCP to extend agent capabilities through external tools and services.

MCP enables communication between agents and MCP servers that provide additional tools. The Strands Agent SDK includes built-in support for connecting to MCP servers and using their tools.

In this example we will show you how to use MCP tools on your Strands Agent. We will use the [AWS Documentation MCP server](https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server/) which provides tools to access AWS documentation, search for content, and get recommendations. This MCP server has 3 main features:

- **Read Documentation**: Fetch and convert AWS documentation pages to markdown format
- **Search Documentation**: Search AWS documentation using the official search API
- **Recommendations**: Get content recommendations for AWS documentation pages



## Agent Details
<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|Feature used        |MCP Tools                                          |
|Agent Structure     |Single agent architecture                          |

</div>

## Architecture

<div style="text-align:center">
    <img src="images/architecture.png" width="65%" />
</div>

## Key Features
* **Single agent architecture**: this example creates a single agent that interacts with MCP tools
* **MCP tools**: Integration of MCP tools with your agent

## Setup and prerequisites

### Prerequisites
* Python 3.10+
* AWS account
* Anthropic Claude 3.7 enabled on Amazon Bedrock

Let's now install the requirement packages for our Strands Agent agent

In [1]:
# installing pre-requisites
!uv pip install -r requirements.txt

Using Python 3.13.5 environment at: e:\OneDrive\OneDriveOnitbuddy\OneDrive\workbench\exampletest\agenticAIeducation\.venv
Resolved 76 packages in 504ms
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 1 package in 370ms
 + uv==0.8.11


### Importing dependency packages

Now let's import the dependency packages

In [2]:
import threading
import time
from datetime import timedelta

from mcp import StdioServerParameters, stdio_client
from mcp.client.streamable_http import streamablehttp_client
from mcp.server import FastMCP
from strands import Agent
from strands.tools.mcp import MCPClient


### Connect to MCP server using stdio transport

[Transports](https://modelcontextprotocol.io/specification/2025-03-26/basic/transports) in MCP provide the foundations for communication between clients and servers. It handles the underlying mechanics of how messages are sent and received. At the moment there are three standards transport implementations built-in in MCP:

- **Standard Input/Output (stdio)**: enables communication through standard input and output streams. It is particularly useful for local integrations and command-line tools
- **Streamable HTTP**: In the Streamable HTTP transport, the server operates as an independent process that can handle multiple client connections. This transport uses HTTP POST and GET requests. Server can optionally make use of Server-Sent Events (SSE) to stream multiple server messages. This permits basic MCP servers, as well as more feature-rich servers supporting streaming and server-to-client notifications and requests.
- **SSE**: legacy transport for HTTP-based MCP servers that use Server-Sent Events transport  

Overall, you should use stdio for building command-line tools, implementing local integrations and working with shell scripts. You should use Streamable HTTP transports when you need a flexible and efficient way for AI agents to communicate with tools and services, especially when dealing with stateless communication or when minimizing resource usage is crucial.

You can also use **custom transports** implementation for your specific needs. 


Let's now connect to the MCP server using stdio transport. First of all, we will use the class `MCPClient` to connect to the [AWS Documentation MCP Server](https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server/). This server provides tools to access AWS documentation, search for content, and get recommendations.

In [3]:
# Connect to an MCP server using stdio transport
stdio_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

#### Setup agent configuration and invoke it

Next we will set our agent configuration using the tools from the `stdio_mcp_client` object we just created. To do so, we need to list the tools available in the MCP server. We can use the `list_tools_sync` method for it. 

After that, we will ask a question to our agent.

In [4]:
# Create an agent with MCP tools
with stdio_mcp_client:
    # Get the tools from the MCP server
    tools = stdio_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(
        model="apac.anthropic.claude-3-7-sonnet-20250219-v1:0",
        tools=tools)

    response = agent("What is Amazon Bedrock pricing model. Be concise.")

client failed to initialize
Traceback (most recent call last):
  File "C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\os\win32\utilities.py", line 169, in create_windows_process
    process = await anyio.open_process(
              ^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<8 lines>...
    )
    ^
  File "C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\anyio\_core\_subprocesses.py", line 190, in open_process
    return await get_async_backend().open_process(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<11 lines>...
    )
    ^
  File "C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\anyio\_backends\_asyncio.py", line 2561, in open_process
    process = await asyncio.create_subprocess_exec(
              ^^^^^

MCPClientInitializationError: the client initialization failed

### Connect to MCP server using Streamable HTTP

Let's now connect to the MCP server using Streamable HTTP transport. First let's start a simple MCP server using Streamable HTTP transport. 

For this example we will create our own MCP server. The architecture will look as following

<div style="text-align:center">
    <img src="images/architecture_2.png" width="65%" />
</div>

In [5]:
# Create an MCP server
mcp = FastMCP("Calculator Server")

# Define a tool
@mcp.tool(description="Calculator tool which performs calculations")
def calculator(x: int, y: int) -> int:
    return x + y

@mcp.tool(description="This is a long running tool")
def long_running_tool(name: str) -> str:
    time.sleep(25)
    return f"Hello {name}"


def main():
    mcp.run(transport="streamable-http", mount_path="mcp")

Let's now start a thread with the `streamable-http` server

In [ ]:
# Thread object is a Python threading mechanism that runs the main() function in a separate background thread.
# This is no longer required if its deployed in a Lambda or Docker container
thread = threading.Thread(target=main)
thread.start()

INFO:     Started server process [30348]
INFO:     Waiting for application startup.


[08/15/25 11:21:10] INFO     StreamableHTTP session manager started                  ]8;id=818011;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\server\streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=549764;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\server\streamable_http_manager.py#110\110]8;;\

INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:62684 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:62690 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:62689 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:62692 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:62699 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:62701 - "DELETE /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:62709 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:62715 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:62714 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:62717 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:62719 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:62721 - "DELETE /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:62739 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:62745 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:62744 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:62747 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:62752 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:62754 - "

#### Integrating Streamable HTTP client with Agent

Now let's use `streamablehttp_client` integrate this server with a simple agent. 

In [7]:
def create_streamable_http_transport():
    return streamablehttp_client("http://localhost:8000/mcp")


streamable_http_mcp_client = MCPClient(create_streamable_http_transport)

#### Setup agent configuration and invoke it

Next we will set our agent configuration using the tools from the `streamable_http_mcp_client` object we just created. To do so, we need to list the tools available in the MCP server. We can use the `list_tools_sync` method for it. 

After that, we will ask a question to our agent.

In [8]:
with streamable_http_mcp_client:
    tools = streamable_http_mcp_client.list_tools_sync()

    agent = Agent(
        model="apac.anthropic.claude-3-7-sonnet-20250219-v1:0",
        tools=tools)

    response = str(agent("What is 2 + 2?"))

[08/15/25 11:22:48] INFO     Created new transport with session ID:                  ]8;id=277002;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\server\streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=465160;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\server\streamable_http_manager.py#233\233]8;;\
                             9b249d0902ab45ac9418bf825d2bef40                                                      

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=124543;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=253817;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\httpx\_client.py#1740\1740]8;;\

                    INFO     Received session ID: 9b249d0902ab45ac9418bf825d2bef40           ]8;id=155789;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\client\streamable_http.py\streamable_http.py]8;;\:]8;id=741649;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\client\streamable_http.py#134\134]8;;\

                    INFO     Negotiated protocol version: 2025-06-18                         ]8;id=792069;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\client\streamable_http.py\streamable_http.py]8;;\:]8;id=822237;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\client\streamable_http.py#146\146]8;;\

                    INFO     HTTP Request: GET http://localhost:8000/mcp "HTTP/1.1 200 OK"          ]8;id=529428;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=318578;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\httpx\_client.py#1740\1740]8;;\

...

[08/15/25 11:22:52] INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=472806;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=601333;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\httpx\_client.py#1740\1740]8;;\

                    INFO     Processing request of type CallToolRequest                               ]8;id=987675;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\server\lowlevel\server.py\server.py]8;;\:]8;id=262987;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\server\lowlevel\server.py#625\625]8;;\

The answer to 2 + 2 is 4.

[08/15/25 11:22:53] INFO     Terminating session: 9b249d0902ab45ac9418bf825d2bef40           ]8;id=933508;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\server\streamable_http.py\streamable_http.py]8;;\:]8;id=433846;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\server\streamable_http.py#630\630]8;;\

                    INFO     HTTP Request: DELETE http://localhost:8000/mcp "HTTP/1.1 200 OK"       ]8;id=342320;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=270844;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\httpx\_client.py#1740\1740]8;;\

### Direct Tool Invocation

While tools are typically invoked by the agent based on user requests, you can also call MCP tools directly. This can be useful for workflow scenarios where you orchestrate multiple tools together.

In [9]:
query = {"x": 10, "y": 20}

with streamable_http_mcp_client:
    # direct tool invocation
    result = streamable_http_mcp_client.call_tool_sync(
        tool_use_id="tool-123", name="calculator", arguments=query
    )

    # Process the result
    print(f"Calculation result: {result['content'][0]['text']}")

[08/15/25 11:23:40] INFO     Created new transport with session ID:                  ]8;id=647050;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\server\streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=765019;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\server\streamable_http_manager.py#233\233]8;;\
                             c652c695b8424881aa898014a97fd24b                                                      

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=717067;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=227851;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\httpx\_client.py#1740\1740]8;;\

                    INFO     Received session ID: c652c695b8424881aa898014a97fd24b           ]8;id=488782;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\client\streamable_http.py\streamable_http.py]8;;\:]8;id=103096;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\client\streamable_http.py#134\134]8;;\

                    INFO     Negotiated protocol version: 2025-06-18                         ]8;id=707771;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\client\streamable_http.py\streamable_http.py]8;;\:]8;id=926669;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\client\streamable_http.py#146\146]8;;\

                    INFO     HTTP Request: GET http://localhost:8000/mcp "HTTP/1.1 200 OK"          ]8;id=846724;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=183556;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\httpx\_client.py#1740\1740]8;;\

...

[08/15/25 11:23:41] INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=215829;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=888944;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\httpx\_client.py#1740\1740]8;;\

                    INFO     Processing request of type ListToolsRequest                              ]8;id=563847;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\server\lowlevel\server.py\server.py]8;;\:]8;id=335563;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\server\lowlevel\server.py#625\625]8;;\

Calculation result: 30


                    INFO     Terminating session: c652c695b8424881aa898014a97fd24b           ]8;id=129941;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\server\streamable_http.py\streamable_http.py]8;;\:]8;id=242604;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\server\streamable_http.py#630\630]8;;\

                    INFO     HTTP Request: DELETE http://localhost:8000/mcp "HTTP/1.1 200 OK"       ]8;id=617257;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=823548;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\httpx\_client.py#1740\1740]8;;\

You can optionally also provide `read_timeout_seconds` while calling an MCP server tool to avoid it running for too long

In [10]:
with streamable_http_mcp_client:
    try:
        result = streamable_http_mcp_client.call_tool_sync(
            tool_use_id="tool-123",
            name="long_running_tool",
            arguments={"name": "AutoGuru"},
            read_timeout_seconds=timedelta(seconds=30),
        )

        if result["status"] == "error":
            print(f"Tool execution failed: {result['content'][0]['text']}")
        else:
            print(f"Tool execution succeeded: {result['content'][0]['text']}")
    except Exception as e:
        print(f"Tool call timed out or failed: {str(e)}")

[08/15/25 11:24:45] INFO     Created new transport with session ID:                  ]8;id=534359;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\server\streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=599156;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\server\streamable_http_manager.py#233\233]8;;\
                             6388c05c60654317a960e6115f468006                                                      

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=772141;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=132413;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\httpx\_client.py#1740\1740]8;;\

                    INFO     Received session ID: 6388c05c60654317a960e6115f468006           ]8;id=63008;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\client\streamable_http.py\streamable_http.py]8;;\:]8;id=960730;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\client\streamable_http.py#134\134]8;;\

                    INFO     Negotiated protocol version: 2025-06-18                         ]8;id=526145;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\client\streamable_http.py\streamable_http.py]8;;\:]8;id=201492;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\client\streamable_http.py#146\146]8;;\

[08/15/25 11:24:46] INFO     HTTP Request: GET http://localhost:8000/mcp "HTTP/1.1 200 OK"          ]8;id=742396;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=803192;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\httpx\_client.py#1740\1740]8;;\

...

[08/15/25 11:25:11] INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=382899;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=577321;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\httpx\_client.py#1740\1740]8;;\

                    INFO     Processing request of type ListToolsRequest                              ]8;id=107776;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\server\lowlevel\server.py\server.py]8;;\:]8;id=113300;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\server\lowlevel\server.py#625\625]8;;\

Tool execution succeeded: Hello AutoGuru


                    INFO     Terminating session: 6388c05c60654317a960e6115f468006           ]8;id=730052;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\server\streamable_http.py\streamable_http.py]8;;\:]8;id=183316;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\mcp\server\streamable_http.py#630\630]8;;\

                    INFO     HTTP Request: DELETE http://localhost:8000/mcp "HTTP/1.1 200 OK"       ]8;id=723161;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=89657;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\httpx\_client.py#1740\1740]8;;\

### Interacting with multiple MCP servers

With Strands Agents you can also interact with multiple MCP servers using the same agent and configure tools setups such as the max number of tools that can be used in parallel (`max_parallel_tools`). Let's create a new agent to showcase this configuration:

<div style="text-align:center">
    <img src="images/architecture_3.png" width="85%" />
</div>

In this agent, we will again use the AWS Documentation MCP server and we will also use the [AWS CDK MCP Server](https://awslabs.github.io/mcp/servers/cdk-mcp-server/) which helps with AWS Cloud Development Kit (CDK) best practices, infrastructure as code patterns and security compliance with CDK Nag.

First let's connect to the two MCP servers using the stdio transport

In [11]:
# Connect to an MCP server using stdio transport
aws_docs_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

# Connect to an MCP server using stdio transport
cdk_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(command="uvx", args=["awslabs.cdk-mcp-server@latest"])
    )
)

#### Create Agent with MCP servers

Next we will create the agent with the tools from both MCP servers

In [12]:
# Create an agent with MCP tools
with aws_docs_mcp_client, cdk_mcp_client:
    # Get the tools from the MCP server
    tools = aws_docs_mcp_client.list_tools_sync() + cdk_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(
        model="apac.anthropic.claude-3-7-sonnet-20250219-v1:0",
        tools=tools)

    response = agent(
        "What is Amazon Bedrock pricing model. Be concise. Also what are the best practices related to CDK?"
    )

[08/15/25 11:27:51] ERROR    client failed to initialize                                          ]8;id=631017;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\strands\tools\mcp\mcp_client.py\mcp_client.py]8;;\:]8;id=955071;file://C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\strands\tools\mcp\mcp_client.py#115\115]8;;\
                             ╭─────────────── Traceback (most recent call last) ────────────────╮                  
                             │ C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.P │                  
                             │ ython.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\sit │                  
                             │ e-packages\mcp\os\win32\utilities.py:169 in                      │                  
                             │ create_windows_process                                           │                  
                             │                                                                  │                  
                             │   166 │                                                          │                  
                             │   167 │   try:                                                   │                  
                             │   168 │   │   # First try using anyio with Windows-specific flag │                  
                             │ ❱ 169 │   │   process = await anyio.open_process(                │                  
                             │   170 │   │   │   [command, *args],                              │                  
                             │   171 │   │   │   env=env,                                       │                  
                             │   172 │   │   │   # Ensure we don't create console windows for e │                  
                             │                                                                  │                  
                             │ C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.P │                  
                             │ ython.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\sit │                  
                             │ e-packages\anyio\_core\_subprocesses.py:190 in open_process      │                  
                             │                                                                  │                  
                             │   187 │   if umask >= 0:                                         │                  
                             │   188 │   │   kwargs["umask"] = umask                            │                  
                             │   189 │                                                          │                  
                             │ ❱ 190 │   return await get_async_backend().open_process(         │                  
                             │   191 │   │   command,                                           │                  
                             │   192 │   │   stdin=stdin,                                       │                  
                             │   193 │   │   stdout=stdout,                                     │                  
                             │                                                                  │                  
                             │ C:\Users\mnede\AppData\Local\Packages\PythonSoftwareFoundation.P │                  
                             │ ython.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\sit │                  
                             │ e-packages\anyio\_backends\_asyncio.py:2561 in open_process      │                  
                             │                                                                  │                  


MCPClientInitializationError: the client initialization failed

### Congratulations!

In this notebook you learned how to connect with MCP servers using Strands Agent and two MCP transport protocols: stdio and Streamable HTTP. You also learned how to connect multiple MCP servers to the same agent. Next, let's see how to use different models with your agent